In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.5.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/mingdongfan/anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.11.29         |        py37_1000         145 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    pyhamcrest-1.9.0           |             py_2          23 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    twisted-18.9.0             |   py37h470a237_0         4.9 MB  conda-forge
    conda-4.5.12               |        py37_1000         652 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB

## 1. Toronto Neighbourhoods

In [4]:
# Toronto Neighbourhoods Data
G3 = pd.read_csv('G3.csv')
G3 = G3.drop('Unnamed: 0', axis = 1)
G3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Toronto Neighbourhoods Map

In [18]:
# create map of Manhattan using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(G3['Latitude'], G3['Longitude'], G3['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'QS44NWUNPK5JSK4G5XJFUGL1VJNCDP2MGBQ5NDOVDPJAANHK' # your Foursquare ID
CLIENT_SECRET = 'XKVRUXX4RTZZKGGWHT03AOHKJ3EDI4XMFH5WZYCNL5FSJOY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## 2. Venues Near Selected Neighbourhoods

### Select Neighbourhoods with string 'Toronto'

In [24]:
G4 = G3[G3['Neighbourhood'].str.contains('Toronto')]
G4

,Postcode,Borough,Neighbourhood,Latitude,Longitude
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763
40,M4J,East York,East Toronto,43.685347,-79.338106
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
66,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321


### Extract Neighbourhood Venues Using Foursquare

In [56]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
toronto_venues = getNearbyVenues(names=G4['Neighbourhood'], latitudes=G4['Latitude'], longitudes=G4['Longitude'])

CFB Toronto, Downsview East
East Toronto
North Toronto West
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Harbord, University of Toronto
Humber Bay Shores, Mimico South, New Toronto


### Number of Venues

In [58]:
print(toronto_venues.shape)
toronto_venues.head()

(274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CFB Toronto, Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
1,"CFB Toronto, Downsview East",43.737473,-79.464763,Ttc Bus #120 - Plewes Rd,43.734898,-79.464221,Bus Stop
2,"CFB Toronto, Downsview East",43.737473,-79.464763,Ancaster Park,43.733724,-79.465528,Park
3,East Toronto,43.685347,-79.338106,The Path,43.683923,-79.335007,Park
4,East Toronto,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store


In [60]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CFB Toronto, Downsview East",3,3,3,3,3,3
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
East Toronto,3,3,3,3,3,3
"Harbord, University of Toronto",33,33,33,33,33,33
"Harbourfront East, Toronto Islands, Union Station",100,100,100,100,100,100
"Humber Bay Shores, Mimico South, New Toronto",16,16,16,16,16,16
North Toronto West,19,19,19,19,19,19


In [61]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 98 uniques categories.


## 3. Analyze Each Neighborhood

### Frequency of Venues Grouped by neighborhood

In [62]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [63]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Event Space,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food Court,French Restaurant,Fried Chicken Joint,Gastropub,General Travel,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Lounge,Mexican Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Poutine Place,Pub,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,"CFB Toronto, Downsview East",0.000000,0.333333,0.0000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.0000,0.00,0.000000,0.0000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.333333,0.00,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
1,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.0400,0.00,0.01,0.01,0.010000,0.00,0.020000,0.00,0.01,0.010000,0.000000,0.00,0.010000,0.00,0.00,0.02,0.000000,0.090000,0.000000,0.000000,0.140000,0.000000,0.000000,0.000000,0.02,0.000000,0.00,0.03,0.000000,0.000000,0.00,0.010000,0.0000,0.0000,0.01,0.010000,0.0100,0.03,0.01,0.01,0.01,0.000000,0.030000,0.01,0.00,0.07,0.01,0.01,0.00,0.00,0.020000,0.020000,0.00,0.0000,0.01,0.000000,0.00,0.01,0.00,0.01,0.000000,0.010000,0.01,0.000000,0.00,0.0000,0.0100,0.00,0.000000,0.010000,0.000000,0.050000,0.02,0.000000,0.010000,0.00,0.02,0.01,0.00,0.00,0.01,0.000000,0.01,0.010000,0.02,0.02,0.00,0.000000,0.01,0.02,0.010000,0.01,0.00,0.000000,0.01
2,East Toronto,0.000000,0.000000,0.0000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.333333,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.0000,0.00,0.000000,0.0000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.333333,0.00,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
3,"Harbord, University of Toronto",0.000000,0.000000,0.0000,0.00,0.00,0.00,0.060606,0.00,0.060606,0.00,0.00,0.030303,0.030303,0.00,0.060606,0.00,0.00,0.00,0.000000,0.121212,0.030303,0.000000,0.060606,0.030303,0.030303,0.030303,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.0000,0.00,0.030303,0.0000,0.00,0.00,0.00,0.00,0.000000,0.030303,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.030303,0.060606,0.00,0.0000,0.00,0.000000,0.00,0.00,0.00,0.00,0.030303,0.030303,0.00,0.000000,0.00,0.0000,0.0000,0.00,0.030303,0.030303,0.000000,0.060606,0.00,0.000000,0.030303,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0

### Top 10 venues for each neighborhood

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Wine Bar,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner
1,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Gym,Bar,Steakhouse
2,East Toronto,Coffee Shop,Convenience Store,Park,Wine Bar,Fish & Chips Shop,Concert Hall,Dance Studio,Deli / Bodega,Dessert Shop,Diner
3,"Harbord, University of Toronto",Café,Japanese Restaurant,Coffee Shop,Bakery,Bar,Restaurant,Bookstore,Italian Restaurant,Nightclub,Noodle House
4,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Restaurant,Bakery,Scenic Lookout,Italian Restaurant,Brewery
5,"Humber Bay Shores, Mimico South, New Toronto",Coffee Shop,Flower Shop,Gym,Restaurant,Café,Sandwich Place,Fast Food Restaurant,Pharmacy,Fish & Chips Shop,Liquor Store
6,North Toronto West,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Spa,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park


## 4. Cluster Neighborhoods

### K-means Clustering

In [89]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 0, 0, 4, 3], dtype=int32)

In [91]:
toronto_merged = G4
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

/Users/mingdongfan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,1,Airport,Park,Bus Stop,Wine Bar,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner
40,M4J,East York,East Toronto,43.685347,-79.338106,0,Coffee Shop,Convenience Store,Park,Wine Bar,Fish & Chips Shop,Concert Hall,Dance Studio,Deli / Bodega,Dessert Shop,Diner
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Spa,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Restaurant,Bakery,Scenic Lookout,Italian Restaurant,Brewery
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Gym,Bar,Steakhouse


In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters